# Model Selection 

The objective of this notebook is to build a very first ent to end Machine Learning model to predict the probability of a patient being discharged on a particular day.

We'll use the dataset generated by the `dataset` job of the ETL folder. 

At this instance, the focus won't be on model performance but rather on understanding the value potential of the available data and the speedness of the solution.

In [1]:
%cd /Users/josefinadallavia/Documents/MIM/Tesis/AML-hospital

/Users/josefinadallavia/Documents/MIM/Tesis/AML-hospital


In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from matplotlib import pyplot as plt   
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from thesis_lib.utils import * 
from thesis_lib.modelling.data import *
from thesis_lib.modelling.model import *


/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
data = Data().load('data/hospital_dataset')
variables = data.get_variables_dict()
data.get_stats()

Loading dataset:  hospital_train_data.parquet
Loading dataset:  hospital_val_data.parquet
Loading dataset:  hospital_test_data.parquet


dataset_type,train,val,test
n_observations,319150,33482,33309
relative_size,0.82694,0.0867542,0.0863059
n_cols,71,71,71
positives,42697,4555,4507
negatives,276453,28927,28802
positive_prop,0.133783,0.136043,0.135309
negative_prop,0.866217,0.863957,0.864691
min_date,2017-01-01,2018-11-11,2018-11-11
max_date,2018-11-10,2019-11-11,2019-11-11


In [4]:
sequence_features = ['labos_studies_names','sectors_names','images_study_types','surgeries_types',
                    'surgeries_injury_condition','surgeries_post_surgery_condition','surgeries_anesthesia_types']

In [5]:
text_features = ['administrative_diagnosis','presumptive_dianogsis','images_studies_names',
                'surgeries_actual']

In [6]:
text_features = [ 'administrative_diagnosis','presumptive_dianogsis','images_studies_names',
                'surgeries_actual' ,            
    'labos_studies_names','sectors_names','images_study_types','surgeries_types',
                    'surgeries_injury_condition','surgeries_post_surgery_condition','surgeries_anesthesia_types']

In [7]:
categorical_features = ['date_weekday',
 'request_origin',
 'origin',
 'entity_group',
 'gender',
 'request_sector',
 'insurance_entity',
 'admission_sector',
 'emergency_service',
 'isolation',
 'ARM_TEP',
 'CEC_TEP',
 'request_sector',
 'date_weekday',
 'admission_weekday',
 'date_month',
 'admission_month']

In [8]:
numerical_features = ['PIM2TEP',
 'emergency_service',
 'high_risk_TEP',
 'hosp_day_number',
 'images_count',
 'images_cumulative',
 'images_emergencies',
 'images_requester_roles_count',
 'images_requesters_count',
 'images_study_types_count',
 'labos_count',
 'labos_cumulative',
 'labos_emergencies',
 'labos_requester_roles_count',
 'labos_requesters_count',
 'labos_set_count',
 'labos_set_cumulative',
 'low_risk_TEP',
 'new_born_gestation_age',
 'new_born_weight',
 'patient_age',
 'sectors_count',
 'surgeries_count',
 'surgeries_cumulative',
 'surgeries_post_surgery_duration',
 'surgeries_pre_surgery_duration',
 'surgeries_prep_duration',
 'surgeries_services_count',
 'surgeries_surgery_delay',
 'surgeries_surgery_duration',
 'surgeries_types_count']

### All features

In [10]:
model_params = {'classifier': 'random_forest',
               'accepts_sparse': True,
                'categorical_features' : categorical_features ,
                'numerical_features' : numerical_features,
               'text_features': text_features,
                #'sequence_features':sequence_features
               }

In [11]:
rf_comb_features = Model(**model_params)
rf_comb_features.transform(data)

Fitting pipeline...
	 Preprocessing data
	 Preprocessing data
	 Encoding Categorical Features with OneHotEncoding
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
Transforming data...
	 Transforming numerical features
	 Preprocessing data
	 Transforming categorical features
	 Preprocessing data
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transformi

In [11]:
rf_comb_features.n_features

20090

### Fine tuning

### Objective function

In [12]:
import lightgbm as lgb
import flatdict
from hyperopt import STATUS_OK

N_FOLDS = 3

# Create the dataset
train_set = lgb.Dataset(rf_comb_features.X_train,label=rf_comb_features.y_train)

def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    flatten_params = flatdict.FlatDict(params)
    parameters ={param_name: flatten_params[param_name] for param_name in flatten_params}
    
    
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    cv_results = lgb.cv(params = parameters, train_set = train_set,
                        nfold = n_folds,
                        early_stopping_rounds = 100, metrics = 'auc', seed = 2020,
                        verbose_eval=True)
  
    # Extract the best score
    best_score = max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

### Domain space

In [13]:
from hyperopt import hp

space_rf = {
    'objective': hp.choice('objective', ['binary']),
  
    'boosting_type': 'rf', 
    
    'bagging_freq': hp.quniform('bagging_freq', 1,20,1), 
    
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 0.99),
    
    'max_depth': hp.quniform('max_depth', 5, 30,1),
    
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    
    'num_iterations': hp.choice('num_iterations', [100,150,200,300,500])
}

In [14]:
from hyperopt import hp

space = {
    'objective': hp.choice('objective', ['binary']),
  
    'boosting_type': hp.choice('boosting_type', 
                               [{'boosting_type': 'rf', 
                                     'bagging_freq': hp.quniform('rf_bagging_freq', 1, 10,1), 
                                     'bagging_fraction': hp.uniform('rf_bagging_fraction', 0.5, 0.99)
                                }, 
                                
                                {'boosting_type': 'goss',  
                                     'subsample': 1.0 },
                                
                                {'boosting_type': 'gdbt',
                                    'subsample': hp.uniform('gdbt_subsample', 0.5, 1)} 
                               ]),
    'max_depth': hp.quniform('max_depth', 5, 30, 15,1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
     'num_iterations': hp.choice('num_iterations', [100,150,200,300,500])
}

In [15]:
import hyperopt.pyll.stochastic 

### Optimization algorithm

In [16]:
from hyperopt import tpe
# Algorithm
tpe_algorithm = tpe.suggest

### Result history

In [17]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials_rf = Trials()

In [18]:
!pwd

/Users/josefinadallavia/Documents/MIM/Tesis/AML-hospital


### Optimization

In [ ]:
from hyperopt import fmin

MAX_EVALS = 200

# Optimize
best = fmin(fn = objective, space = space_rf, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials_rf)

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	cv_agg's auc: 0.785399 + 0.00366293                
[2]	cv_agg's auc: 0.785399 + 0.00366293                
[3]	cv_agg's auc: 0.794925 + 0.000836551               
[4]	cv_agg's auc: 0.795464 + 0.00116942                
[5]	cv_agg's auc: 0.797008 + 0.000881538               
[6]	cv_agg's auc: 0.796919 + 0.000588159               
[7]	cv_agg's auc: 0.797806 + 0.000533835               
[8]	cv_agg's auc: 0.797804 + 0.000841915               
[9]	cv_agg's auc: 0.798699 + 0.00114033                
[10]	cv_agg's auc: 0.79885 + 0.00115216                
[11]	cv_agg's auc: 0.799097 + 0.00118392               
[12]	cv_agg's auc: 0.799058 + 0.00140638               
[13]	cv_agg's auc: 0.799128 + 0.00133775               
[14]	cv_agg's auc: 0.799031 + 0.00124974               
[15]	cv_agg's auc: 0.799299 + 0.00147911               
[16]	cv_agg's auc: 0.799256 + 0.00166288               
[17]	cv_agg's auc: 0.79931 + 0.00135492                
[18]	cv_agg's auc: 0.799125 + 0.00116375        

[147]	cv_agg's auc: 0.800272 + 0.000945707             
[148]	cv_agg's auc: 0.800301 + 0.000909153             
[149]	cv_agg's auc: 0.800323 + 0.000882603             
[150]	cv_agg's auc: 0.80018 + 0.000825485              
[151]	cv_agg's auc: 0.800286 + 0.00086734              
[152]	cv_agg's auc: 0.800252 + 0.000896063             
[153]	cv_agg's auc: 0.80028 + 0.000856148              
[154]	cv_agg's auc: 0.800286 + 0.00086159              
[155]	cv_agg's auc: 0.80032 + 0.000885356              
[156]	cv_agg's auc: 0.800323 + 0.00092125              
[157]	cv_agg's auc: 0.800299 + 0.000960654             
[158]	cv_agg's auc: 0.800302 + 0.000964701             
[159]	cv_agg's auc: 0.800289 + 0.000962064             
[160]	cv_agg's auc: 0.800239 + 0.000838605             
[161]	cv_agg's auc: 0.800231 + 0.00085581              
[162]	cv_agg's auc: 0.800346 + 0.000926359             
[163]	cv_agg's auc: 0.800322 + 0.000947467             
[164]	cv_agg's auc: 0.800311 + 0.000946647      

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



...

[185]	cv_agg's auc: 0.794485 + 0.00301864                                               
[186]	cv_agg's auc: 0.79446 + 0.0029969                                                 
[187]	cv_agg's auc: 0.794466 + 0.00300312                                               
[188]	cv_agg's auc: 0.794463 + 0.00300059                                               
[189]	cv_agg's auc: 0.794385 + 0.00293008                                               
[190]	cv_agg's auc: 0.794387 + 0.00291997                                               
[191]	cv_agg's auc: 0.794659 + 0.00300836                                               
[192]	cv_agg's auc: 0.794681 + 0.00302227                                               
[193]	cv_agg's auc: 0.794699 + 0.00303843                                               
[194]	cv_agg's auc: 0.794752 + 0.00311508                                               
[195]	cv_agg's auc: 0.794848 + 0.0031222                                                
[196]	cv_agg's auc: 0

[277]	cv_agg's auc: 0.795241 + 0.00315114                                               
[278]	cv_agg's auc: 0.795238 + 0.00315482                                               
[279]	cv_agg's auc: 0.795248 + 0.00316713                                               
[280]	cv_agg's auc: 0.795261 + 0.00319188                                               
[281]	cv_agg's auc: 0.795249 + 0.00318943                                               
[282]	cv_agg's auc: 0.795245 + 0.00319028                                               
[283]	cv_agg's auc: 0.795275 + 0.00321634                                               
[284]	cv_agg's auc: 0.795284 + 0.00322832                                               
[285]	cv_agg's auc: 0.795273 + 0.00320838                                               
[286]	cv_agg's auc: 0.795295 + 0.00323251                                               
[287]	cv_agg's auc: 0.795292 + 0.00322583                                               
[288]	cv_agg's auc: 0

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	cv_agg's auc: 0.786762 + 0.00335024                                                 
[2]	cv_agg's auc: 0.794123 + 0.00135421                                                 
[3]	cv_agg's auc: 0.79743 + 0.000360247                                                 
[4]	cv_agg's auc: 0.797777 + 0.000406726                                                
[5]	cv_agg's auc: 0.797912 + 0.000845699                                                
[6]	cv_agg's auc: 0.798259 + 0.000834361                                                
[7]	cv_agg's auc: 0.798304 + 0.00094148                                                 
[8]	cv_agg's auc: 0.798288 + 0.00103454                                                 
[9]	cv_agg's auc: 0.798373 + 0.00121593                                                 
[10]	cv_agg's auc: 0.798709 + 0.000693878                                               
[11]	cv_agg's auc: 0.798947 + 0.000796867                                               
[12]	cv_agg's auc: 0.

[93]	cv_agg's auc: 0.800227 + 0.00082037                                                
[94]	cv_agg's auc: 0.800176 + 0.00086172                                                
[95]	cv_agg's auc: 0.800207 + 0.00091315                                                
 32%|███▏      | 63/200 [4:26:38<9:23:21, 246.73s/trial, best loss: 0.19752737237658413]

In [ ]:
bayes_trials_rf.results